In [ ]:
% run 1-datasource.ipynb

In [ ]:
import torch
from torch import nn
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import scipy.stats as sc
import torch.nn.functional as F
from skorch.net import NeuralNet,train_loss_score,valid_loss_score
from skorch.callbacks import EpochTimer,EpochScoring,PrintLog
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler_1 = StandardScaler()
scaler_2 = StandardScaler()

In [ ]:
for _, i in icebergs.iterrows():
    scaler_1.partial_fit(i.band_1.reshape(1, -1))
    scaler_2.partial_fit(i.band_2.reshape(1, -1))

In [ ]:
y = icebergs.is_iceberg.values.astype(np.float32)
x = (
    np.stack(
        [
            scaler_1.transform(np.stack(icebergs.band_1)),
            scaler_2.transform(np.stack(icebergs.band_2)),
        ],
        axis=1
    ).reshape(-1, 2, 75, 75)
).astype(np.float32)

In [ ]:
x.dtype, y.dtype

In [ ]:
type(x), type(y)

In [ ]:
y.shape, x.shape

In [ ]:
insize1 = 2 
#outsize1 = 32 
#outsize2 = 32

class Net(nn.Module):
    def __init__(self,outsize1=16,outsize2=32):
        super().__init__()
        
        #self.padding = padding
        #self.kernel1 = kernel1
        #self.kernel2 = kernel2 
        self.outsize1= outsize1
        self.outsize2 = outsize2
        #self.stride = stride
        #self.group? 
        #insize outsize ?
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(insize1, outsize1, kernel_size=7, stride=1, padding=2, groups=2),
            nn.BatchNorm2d(outsize1),
            nn.ReLU(),
            nn.MaxPool2d(4)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(outsize1, outsize2, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(outsize2),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(outsize2*81, 1024),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.ReLU(),
            nn.Linear(512, 1),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.fc(out)
        return out

In [ ]:
#redifining scoring for sklearn 
from skorch import NeuralNet
from skorch.utils import to_numpy
class NNplusplus(NeuralNet):
    def score(self,X,target):
        y_preds = []
        for yp in self.forward_iter(X, training=False):
            y_preds.append(to_numpy(yp.sigmoid()))
       
        y_preds = np.concatenate(y_preds, 0)
   
        return log_loss(target,y_preds)

In [ ]:
from skorch.net import NeuralNetClassifier
model = NNplusplus(
    Net,
    optimizer=torch.optim.Adam,
    criterion=nn.BCEWithLogitsLoss, #regularization
    max_epochs=150,
    batch_size=25,
    lr=0.000001,
    use_cuda = True
)

In [ ]:
model.fit(x,y)

In [ ]:
param_dis = {
    'batch_size': [32,64],
    'module__outsize1':[16,32,64],
    'module__outsize2':[16,32,64],
}

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
gsR = RandomizedSearchCV(model, param_dis, refit=False,scoring='f1',n_jobs=1,n_iter=5)

In [ ]:
gsR.fit(x,y)

In [ ]:
gsR.best_params_,gsR.best_score_